## importing libraries

In [ ]:
import pandas as pd
import numpy as np

## loading datasets

In [ ]:
movies = pd.read_csv('./data/movies.csv')
credits = pd.read_csv('./data/credits.csv')
print(movies.shape, credits.shape)
movies.head(1)

In [ ]:
credits.head(2)

In [ ]:
#merge credits with movies dataframe
movies_df = movies.merge(credits, on="title")
print(movies.shape)
movies.head(1)

## Data preprocessing

In [ ]:
#extracting only required columns
movies_df = movies_df.loc[:, ['movie_id','title','genres','overview','keywords','cast','crew']]
movies_df.head(1)

In [ ]:
#check if any missing value present in the dataset
movies_df.isnull().sum()

In [ ]:
# drop rows having missing values
movies_df.dropna(inplace = True)

In [ ]:
# check if any duplicate rows present
movies_df.duplicated().sum()

In [ ]:
movies_df['overview']

In [ ]:
#converting string into list
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [ ]:
movies_df['genres'].values

In [ ]:
import ast

def convert(obj):
    '''
    this function returns the name from the dict 
    '''
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df.head(1)

In [ ]:
movies_df['keywords'] = movies_df['keywords'].apply(convert)

In [ ]:
def convert2(obj):
    '''
    this function returns the first three name from the dict
    '''
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies_df['cast'] = movies_df['cast'].apply(convert2)

In [ ]:
movies_df.head(1)

In [ ]:
def convert_crew(obj):
    '''
    this function returns the name from the dict where the job is Director
    '''
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies_df['crew'] = movies_df['crew'].apply(convert_crew)

In [ ]:
movies_df.head(3)

In [ ]:
#transforming the string having space to nonspace 
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies_df.head(2)

In [ ]:
movies_df['tags'] = movies_df['genres'] + movies_df['overview'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']
movies_df.head(3)

In [ ]:
#taking only necessary columns
new_movies_df = movies_df.loc[:, ['movie_id', 'title', 'tags']]
new_movies_df.head()

In [ ]:
#converting list to string 
new_movies_df['tags'] = new_movies_df['tags'].apply(lambda x: " ".join(x))
new_movies_df['tags'][0]

In [ ]:
#convert all letters to lowercase
new_movies_df['tags'] = new_movies_df['tags'].apply(lambda x: x.lower())
new_movies_df.head()

In [ ]:
#steming the words
import nltk

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
ps.stem("loving")

In [ ]:
def stem(text):
    """
    this stem function is used to stemming the words
    text - words given by the users which needs to be stemmed
    """
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    string = " ".join(y)
    return string

In [ ]:
new_movies_df['tags'] = new_movies_df['tags'].apply(stem)
new_movies_df.head()

### text vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000, stop_words="english")

In [ ]:
vectors = cv.fit_transform(new_movies_df['tags']).toarray()
vectors

In [ ]:
#calculating the cosine similarity between movies
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

## Recommendation

In [ ]:
# function to recommend movie
def recommend(movie):
    movie_index = new_movies_df[new_movies_df['title']==movie].index[0]
    
    #calculate the distances between the movies
    distances = similarity[movie_index]
    
    #sort the distances and take top five similiar movies
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    print(f'Top 5 recommended movies for {movie} are: \n')
    for i in movies_list:
        print(new_movies_df.iloc[i[0]].title)

In [ ]:
# Recommendation
recommend('Batman Begins')